In [1]:
!apt-get install wget


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz


--2024-09-01 18:41:04--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  6.44MB/s    in 19s     

2024-09-01 18:41:24 (4.23 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [3]:
!tar -xf aclImdb_v1.tar.gz


In [4]:
import os
import pandas as pd

def load_imdb_data(directory):
    data = {'review': [], 'sentiment': []}

    for label in ['pos', 'neg']:
        sentiment_dir = os.path.join(directory, label)
        for review_file in os.listdir(sentiment_dir):
            with open(os.path.join(sentiment_dir, review_file), 'r', encoding='utf-8') as file:
                data['review'].append(file.read())
                data['sentiment'].append(1 if label == 'pos' else 0)

    return pd.DataFrame(data)
train_data = load_imdb_data('aclImdb/train')
test_data = load_imdb_data('aclImdb/test')

train_data.head()


,review,sentiment
0,This movie took the Jerry Springer approach to...,1
1,Twisted Desire (1996) was a TV movie starring ...,1
2,Other reviewers have summarized this film noir...,1
3,"My wife, Kate and I absolutely loved the serie...",1
4,A wonderful early musical film from Rene Clair...,1


In [5]:
len(train_data)

25000

In [6]:
len(test_data)

25000

In [7]:
train_data['review'][24995]

'This anime series starts out great: Interesting story, exciting events, interesting characters, beautifully rendered and executed. Not everything is explained right away, dangling a proverbial carrot before the viewer, enticing the viewer to watch each succeeding episode. But imagine the disappointment to find that the sci-fi thriller/giant robot adventure is only a backdrop for psycho-babble and quasi-religious preachy exploitation. If you want to hear "You\'re OK. It\'s good to be you." after being embattled with negative slogans and the characters\' negative emotions, then this is for you. If you want a good sci-fi flick that is simply fun to watch, forget this one. Both the original, and the alternate endings were grossly disappointing to me. All that, AND this movie was too preachy.'

In [8]:
import numpy as np
print(np.sum(len(train_data['review'][i]) for i in range(len(train_data)))//len(train_data))

<ipython-input-8-447537a875e7>:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  print(np.sum(len(train_data['review'][i]) for i in range(len(train_data)))//len(train_data))


1325


In [9]:
len(np.unique(train_data['review']))

24904

In [10]:
df = train_data.drop_duplicates(subset=['review'])


In [11]:
len(df)

24904

In [12]:
df

,review,sentiment
0,This movie took the Jerry Springer approach to...,1
1,Twisted Desire (1996) was a TV movie starring ...,1
2,Other reviewers have summarized this film noir...,1
3,"My wife, Kate and I absolutely loved the serie...",1
4,A wonderful early musical film from Rene Clair...,1
...,...,...
24995,This anime series starts out great: Interestin...,0
24996,Do you like stand up? Then stay away from this...,0
24997,"In the ravaged wasteland of the future, mankin...",0
24998,This is another typical unbelievable and non-s...,0


In [13]:
df['sentiment'].value_counts()

,count
sentiment,
1,12472
0,12432


In [14]:
!pip install transformers torch

In [15]:
df

,review,sentiment
0,This movie took the Jerry Springer approach to...,1
1,Twisted Desire (1996) was a TV movie starring ...,1
2,Other reviewers have summarized this film noir...,1
3,"My wife, Kate and I absolutely loved the serie...",1
4,A wonderful early musical film from Rene Clair...,1
...,...,...
24995,This anime series starts out great: Interestin...,0
24996,Do you like stand up? Then stay away from this...,0
24997,"In the ravaged wasteland of the future, mankin...",0
24998,This is another typical unbelievable and non-s...,0


In [16]:
from torch.utils.data import Dataset, DataLoader,random_split
import torch.nn as nn
import torch
from transformers import DistilBertTokenizer,DistilBertModel,AdamW


In [17]:
class IMBDDataset(Dataset):
  def __init__(self,df,tokenizer,max_length):
    self.dataset=df
    self.tokenizer=tokenizer
    self.max_length=max_length
  def __len__(self):
    return len(self.dataset)
  def __getitem__(self, index):
    text=self.dataset.iloc[index]['review']
    sentiment=self.dataset.iloc[index]['sentiment']
    encoding=self.tokenizer(text, return_tensors='pt',max_length=self.max_length,truncation=True,padding='max_length')
    # encoding = self.tokenizer.encode_plus(
    #       text,
    #       # add_special_tokens=True,  # Add [CLS] token at the start for classification
    #       max_length=self.max_length,
    #       return_token_type_ids=False,
    #       padding='max_length',
    #       return_attention_mask=True,
    #       return_tensors='pt',
    #       truncation=True
    #     )

    return {
      'review_text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'labels': torch.tensor(self.dataset.iloc[index]['sentiment'], dtype=torch.long)
    }



In [18]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
review_dataset = IMBDDataset(df, tokenizer, 512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [19]:
review_dataset[7770]

{'review_text': 'Let me start off by saying that after watching this episode for the first time on DVD at 10 o\'clock P.M. one night, I could not fall asleep until about 3:00 A.M.<br /><br />This brief review may contain spoilers.<br /><br />I\'m a long-time fan of The Sopranos and I can safely say this is the best episode I\'ve seen. I\'m not saying everyone should feel this way, but I do. This episode is identical to the weekend I spent with my family, watching over my own father, comatose in the ICU before he passed.<br /><br />The episode begins with Tony in an alternate reality: he is a salesman who\'s identity has been mistaken for that of a man named Kevin Finnerty.<br /><br />By the time ten minutes had gone by, I knew either Tony was dreaming, or I was watching some other show. It wasn\'t like the normal Sopranos and I loved it.<br /><br />Option 1 is confirmed when Anthony (or "Kevin") looks into the sky at a "helicopter spotlight" and we see prodding through it, a doctor wit

In [20]:
tokenizer.decode(review_dataset[7770]['input_ids'])

'[CLS] let me start off by saying that after watching this episode for the first time on dvd at 10 o\'clock p. m. one night, i could not fall asleep until about 3 : 00 a. m. < br / > < br / > this brief review may contain spoilers. < br / > < br / > i\'m a long - time fan of the sopranos and i can safely say this is the best episode i\'ve seen. i\'m not saying everyone should feel this way, but i do. this episode is identical to the weekend i spent with my family, watching over my own father, comatose in the icu before he passed. < br / > < br / > the episode begins with tony in an alternate reality : he is a salesman who\'s identity has been mistaken for that of a man named kevin finnerty. < br / > < br / > by the time ten minutes had gone by, i knew either tony was dreaming, or i was watching some other show. it wasn\'t like the normal sopranos and i loved it. < br / > < br / > option 1 is confirmed when anthony ( or " kevin " ) looks into the sky at a " helicopter spotlight " and we

In [21]:
review_dataset[900]

{'review_text': 'TESS OF THE STORM COUNTRY is possibly the best movie of all of Mary Pickford\'s films. At two hours, it was quite long for a 1922 silent film yet continues to hold your interest some 80 years after it was filmed. Mary gives one of her finest performances at times the role seems like a "greatest hits" performance with bits of Mary the innocent, Mary the little devil, Mary the little mother, Mary the spitfire, Mary the romantic heroine, etc. characteristics that often were used throughout a single film in the past. The movie is surprisingly frank about one supporting character\'s illegitimate child for 1922 and at one point our Little Mary is thought the unwed mother in question! If the Academy Awards had been around in 1922, no doubt the Best Actress Oscar for the year would have been Mary\'s.',
 'input_ids': tensor([  101, 15540,  1997,  1996,  4040,  2406,  2003,  4298,  1996,  2190,
          3185,  1997,  2035,  1997,  2984,  4060,  3877,  1005,  1055,  3152,
      

In [22]:
train_size = int(0.8 * len(df))
val_size = len(df) - train_size
train_dataset, test_dataset = random_split(review_dataset, [train_size, val_size])

In [23]:
train_loader=DataLoader(train_dataset,batch_size=10,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=10,shuffle=True)

In [24]:
len(train_loader)+len(test_loader)

2492

In [57]:
class SentimentModel(nn.Module):
  def __init__(self,num_labels=2):
    super().__init__()
    self.distilbert=DistilBertModel.from_pretrained('distilbert-base-uncased')
    self.layer=nn.Linear(768,768)
    self.dropout=nn.Dropout(0.2)
    self.layer2=nn.Linear(768,num_labels)

  def forward(self,input_ids,attention_mask):
    output=self.distilbert(input_ids=input_ids,attention_mask=attention_mask)
    cls=output.last_hidden_state[:,0]
    o1=self.layer(cls)
    o2=nn.ReLU()(o1)
    logits=self.layer2(o2)



    return logits
    # print(output.last_hidden_state.shape)
    # print(output)
    print("*********************")
    # print(output[0]==output.last_hidden_state)




In [26]:
# len(z['input_ids'].flatten())
for i in train_loader:
  z=i
  break

In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
z5=SentimentModel()
z5.to(device)

SentimentModel(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [28]:

for param in z1.distilbert.parameters():
    param.requires_grad = False

trainable_params = sum(p.numel() for p in z1.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")


Number of trainable parameters: 592130


In [50]:
z

{'review_text': ["I was looking on Imdbs bottom 100 because i thought id never seen anything as bad as plan 9 from outerspace or Roller Ball remake, I was wrong. Ben and Arthur has beaten both.<br /><br />This out of the many countless amount of movies I've seen is the number one worst film on the i ever saw. Bad Directing ,Bad Characters ,Horrible Acting ,Horrible story There's a reason nobody but Sam ever says anything positive about this film. Sam was a horrible annoying actor but his directing was so bad he may just overthrow Ed Wood.<br /><br />The Director should be ashamed of his work unfortunately i have to give it at least 1 star but it deserves - to be continued stars.",
  "I.Q., in my opinion, is a sweet, charming, and hilarious romantic comedy about finding the right person for you. If you ask me, James (Stephen Fry) really was a dull guy. To me, Ed (Tim Robbins) was more suited for Catherine (Meg Ryan) than James was. Anyway, everyone involved in this film did an absolutel

In [59]:
y=z5(z['input_ids'].to(device),z['attention_mask'].to(device))

In [60]:
y

tensor([[ 0.0745,  0.1232],
        [ 0.0162,  0.1840],
        [-0.0183,  0.1083],
        [ 0.0288,  0.0507],
        [ 0.0473,  0.1372],
        [ 0.0128,  0.1410],
        [-0.0137,  0.0857],
        [ 0.0414,  0.1212],
        [ 0.0133,  0.1398],
        [ 0.0024,  0.1206]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [30]:
for i in train_loader:
  z=i
  break

In [31]:
print(y)

None


In [32]:
z['input_ids']

tensor([[  101,  1045,  2001,  ...,     0,     0,     0],
        [  101,  1045,  1012,  ...,     0,     0,     0],
        [  101,  2004, 15444,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  6819,  ...,     0,     0,     0],
        [  101,  2520,  8997,  ...,     0,     0,     0],
        [  101,  1045,  2288,  ...,     0,     0,     0]])

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [61]:
z5=SentimentModel()

In [63]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
z5.to(device)

optimizer = AdamW(z5.parameters(), lr=5e-5)

z5.train()
for epoch in range(10):
    for i, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = z5(input_ids=input_ids, attention_mask=attention_mask)
        # predictions = torch.argmax(logits[0], dim=1)
        # print(logits)
        # print(labels)
        loss = nn.CrossEntropyLoss()(logits, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss.item():.4f}")

Epoch 1, Batch 100, Loss: 0.4127
Epoch 1, Batch 200, Loss: 0.3868
Epoch 1, Batch 300, Loss: 0.1545
Epoch 1, Batch 400, Loss: 0.5163
Epoch 1, Batch 500, Loss: 0.2234
Epoch 1, Batch 600, Loss: 0.2963
Epoch 1, Batch 700, Loss: 0.4544
Epoch 1, Batch 800, Loss: 0.1927
Epoch 1, Batch 900, Loss: 0.4092
Epoch 1, Batch 1000, Loss: 0.1203
Epoch 1, Batch 1100, Loss: 0.2370
Epoch 1, Batch 1200, Loss: 0.3181
Epoch 1, Batch 1300, Loss: 0.1556


KeyboardInterrupt: 

In [65]:
z5.eval()
total_correct = 0
total = 0
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    with torch.inference_mode():
        logits = z5(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(logits, dim=1)
    total_correct += (predictions == labels).sum().item()
    total += predictions.size(0)

print(f'Test Accuracy: {total_correct / total:.4f}')

Test Accuracy: 0.9193


In [87]:
def predict_sentiment(text, model, tokenizer, max_length = 512):
    model.eval()
    encoding=tokenizer(text, return_tensors='pt',max_length=max_length,truncation=True,padding='max_length')
    # print(encoding)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.inference_mode():
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
    prediction = torch.argmax(logits, dim=1).item()
    label_dict = {0: 'negative', 1: 'positive'}
    sentiment = label_dict[prediction]
    return sentiment


In [92]:

print(predict_sentiment("that movie was good.", z5, tokenizer))

positive


In [93]:
print(predict_sentiment("that movie was okayish.", z5, tokenizer))

negative
